## はじめに
(1)まずsrc/preprocess/separate_data.pyでデータをクラス毎に分割してください  
 (2)genフォルダをzip形式で圧縮して、gen.zipをgoogle driveにアップロードしてください(場所は任意)  
 (3)gen.zipを右クリックで共有可能なリンクを取得してください  
 (4)下のid変数にリンクの id= 以降を貼り付けてください

In [2]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 20.6MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

下記を実行してリンクを開いてログインユーザーを指定してください  
認証コードをコピーしてEnter verification codeに貼り付け、Enterを押してください

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1bdouVPN6h39sxkRzFMVRClv6AGhMl5oz'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('signate_lunchbox.zip')

In [7]:
!ls

adc.json  gen.zip  sample_data	signate_lunchbox.zip


In [8]:
!unzip signate_lunchbox.zip

Archive:  signate_lunchbox.zip
   creating: signate_lunchbox/
  inflating: signate_lunchbox/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/signate_lunchbox/
  inflating: __MACOSX/signate_lunchbox/._.DS_Store  
   creating: signate_lunchbox/data/
  inflating: signate_lunchbox/data/.DS_Store  
   creating: __MACOSX/signate_lunchbox/data/
  inflating: __MACOSX/signate_lunchbox/data/._.DS_Store  
  inflating: signate_lunchbox/data/test.csv  
  inflating: __MACOSX/signate_lunchbox/data/._test.csv  
  inflating: signate_lunchbox/data/train.csv  
  inflating: __MACOSX/signate_lunchbox/data/._train.csv  


## モデル定義
* 学習済みWideResNetをfine-tuningする
  * keras_contribパッケージが必要になります。 

In [0]:
!pip3 install git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Input
from keras_contrib.applications import WideResidualNetwork

def get_model(input_shape,n_class):
    base_model=WideResidualNetwork(input_shape=input_shape)
    #全結合層の付け替え
    base_model.layers.pop()
    x=base_model.layers[-1].output
    prediction=Dense(n_class,activation='softmax')(x)
    model=Model(inputs=base_model.input,outputs=prediction)
    return model

In [0]:
import os 
import numpy as np
import pandas as pd
from skimage import io
import keras
import tensorflow as tf
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.regularizers import l2
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
import random as rn
from glob import glob

## シード固定
To ensure model reproducibility.

In [0]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
rn.seed(0)
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

train_path='gen/train/'
validation_path='gen/validation/'

## サンプル数の設定
* 訓練と検証のサンプル数を定義している
  * train_ratioはseparate_data.pyと同じ値を使ってください

* 学習時にエポック毎のステップ数(1エポック毎に何回バッチを流すか)の設定が必要(batches per epoch)
  * 通常は全データを見るためにステップ数は(データ数/バッチサイズ)の小数点切り上げにする

In [0]:
the_number_of_images=50000
train_ratio=0.9
train_sample=int(the_number_of_images*train_ratio)
validation_sample=the_number_of_images-train_sample
batch_size=256
train_num_batches_per_epoch=int(the_number_of_images/batch_size)+1
validation_num_batches_per_epoch=int(validation_sample/batch_size)+1


## データ拡張
* ImageDataGeneratorで行いたい前処理(反転、回転、ズーム等)を設定し、インスタンスを作る
* flow_from_directoryを使うことで、指定したパスにある画像セットをImageDataGeneratorで作ったインスタンスによって変換して取り出すことができる

In [0]:
train_data_generator=ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.,
    zoom_range=0.3,
    channel_shift_range=0.1,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1./255
    )
validation_data_generator=train_data_generator=ImageDataGenerator(rescale=1./255)

#data loading
train_generator=train_data_generator.flow_from_directory(
    directory=train_path,
    target_size=(32,32),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical'
)

validation_generator=validation_data_generator.flow_from_directory(
    directory=validation_path,
    target_size=(32,32),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='categorical'
)

Found 45000 images belonging to 20 classes.
Found 5000 images belonging to 20 classes.


## モデル生成

In [0]:
model=get_model(input_shape=(32,32,3),n_class=20)

Instructions for updating:
Colocations handled automatically by placer.
93675520/93671336 [==============================] - 32s 0us/step


## モデルのコンパイル
* 他クラス分類なので誤差関数はカテゴリカルクロスエントロピー
* model.summary()でモデルのレイヤーをプリントできる

In [0]:
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_3 (

## コールバックの定義 
* ModelCheckpoint を使えばエポック毎にモデルを保存してくれる。  
  * さらに if 'save_best_only=True', 'save_weights_only=True' and 'monitor='val_loss'' とすればモデルの重みのみをval_lossが小さくなったエポックだけ保存する。
 * weightsフォルダーに重みが保存されるようになる。


In [0]:
os.makedirs('weights/',exist_ok=True)
mc=keras.callbacks.ModelCheckpoint(os.path.join('weights/','epoch_{epoch:02d}-loss_{val_loss:.2f}.hdf5'), monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

## Training
このコードでは2つのステップでモデルを学習する。  
まずAdam(lr=0.0001)で15エポック学習した後、Adam(lr=0.00001)で10エポック学習する。

In [0]:
first_epoch=15
second_epoch=10

In [0]:
#step 1
history=model.fit_generator(generator=train_generator,
                      steps_per_epoch=train_num_batches_per_epoch,
                      epochs=first_epoch,
                      verbose=1,
                      callbacks=[mc],
                      validation_data=validation_generator,
                      validation_steps=validation_num_batches_per_epoch,
                      class_weight=None,
                      #max_queue_size=10,
                      workers=1,
                      use_multiprocessing=True,
                      shuffle=True,
                      initial_epoch=0)


Instructions for updating:
Use tf.cast instead.
Epoch 1/15
196/196 [==============================] - 315s 2s/step - loss: 2.0015 - acc: 0.3916 - val_loss: 1.5948 - val_acc: 0.4970

Epoch 00001: val_loss improved from inf to 1.59481, saving model to weights/epoch_01-loss_1.59.hdf5
Epoch 2/15
196/196 [==============================] - 298s 2s/step - loss: 1.3802 - acc: 0.5626 - val_loss: 1.3412 - val_acc: 0.5694

Epoch 00002: val_loss improved from 1.59481 to 1.34117, saving model to weights/epoch_02-loss_1.34.hdf5
Epoch 3/15
196/196 [==============================] - 298s 2s/step - loss: 1.1806 - acc: 0.6222 - val_loss: 1.2047 - val_acc: 0.6048

Epoch 00003: val_loss improved from 1.34117 to 1.20465, saving model to weights/epoch_03-loss_1.20.hdf5
Epoch 4/15
196/196 [==============================] - 298s 2s/step - loss: 1.0491 - acc: 0.6638 - val_loss: 1.1272 - val_acc: 0.6266

Epoch 00004: val_loss improved from 1.20465 to 1.12720, saving model to weights/epoch_04-loss_1.13.hdf5
Epoc

In [0]:
#Adamの再定義
opt = Adam(lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

#step1で最もval_lossが小さかった重みを読み込む
weight_path=sorted(glob('weights/*.hdf5'))[-1]
model.load_weights(weight_path)


In [0]:
#step 2
history=model.fit_generator(generator=train_generator,
                      steps_per_epoch=train_num_batches_per_epoch,
                      epochs=first_epoch+second_epoch,
                      verbose=1,
                      callbacks=[mc],
                      validation_data=validation_generator,
                      validation_steps=validation_num_batches_per_epoch,
                      class_weight=None,
                      #max_queue_size=10,
                      workers=1,
                      use_multiprocessing=True,
                      shuffle=True,
                      initial_epoch=first_epoch)

Epoch 16/25
196/196 [==============================] - 305s 2s/step - loss: 0.4670 - acc: 0.8620 - val_loss: 0.9185 - val_acc: 0.7052

Epoch 00016: val_loss did not improve from 0.91090
Epoch 17/25
196/196 [==============================] - 298s 2s/step - loss: 0.4545 - acc: 0.8666 - val_loss: 0.9102 - val_acc: 0.7052

Epoch 00017: val_loss improved from 0.91090 to 0.91016, saving model to weights/epoch_17-loss_0.91.hdf5
Epoch 18/25
196/196 [==============================] - 298s 2s/step - loss: 0.4503 - acc: 0.8690 - val_loss: 0.9096 - val_acc: 0.7066

Epoch 00018: val_loss improved from 0.91016 to 0.90964, saving model to weights/epoch_18-loss_0.91.hdf5
Epoch 19/25
196/196 [==============================] - 298s 2s/step - loss: 0.4407 - acc: 0.8704 - val_loss: 0.9100 - val_acc: 0.7046

Epoch 00019: val_loss did not improve from 0.90964
Epoch 20/25
196/196 [==============================] - 298s 2s/step - loss: 0.4414 - acc: 0.8714 - val_loss: 0.9111 - val_acc: 0.7052

Epoch 00020: va